In [1]:
import pandas as pd
import itertools
import json
text1,text2,labels = [],[],[]
import json

In [2]:
import json
from nezha import Bert,Config
vocab_file = r'/home/xiaoguzai/数据集/Nezha-Base/vocab.txt'
vocab_size = len(open(vocab_file,'r').readlines()) 
with open('/home/xiaoguzai/数据集/Nezha-Base/config.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
json_data['vocab_size'] = vocab_size
print('json_data = ')
print(json_data)
config = Config(**json_data)

json_data = 
{'architectures': ['BertForMaskedLM'], 'attention_probs_dropout_prob': 0.1, 'directionality': 'bidi', 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'model_type': 'bert', 'max_relative_position': 64, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'type_vocab_size': 2, 'vocab_size': 21128, 'use_relative_position': True}


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from tokenization import FullTokenizer
import numpy as np
tokenizer = FullTokenizer(vocab_file=vocab_file)
#预训练程序不需要大改，同一个程序不同的方法实现预训练
from nezha_pretraining import nezha_pretraining_store_id_data

def sequence_padding(inputs,maxlen,padding = 0):
    length = maxlen
    pad_width = [(0,0) for _ in np.shape(inputs[0])]
    #print('pad_width = ')
    #print(pad_width)
    outputs = []
    for x in inputs:
        x = x[:length]
        pad_width[0] = (0,length-len(x))
        x = np.pad(x,pad_width,'constant',constant_values=padding)
        outputs.append(x)
    return np.array(outputs)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
from nezha import Bert
from tqdm import tqdm
import gc
config.with_mlm = True
model = Bert(config)
from loader_nezha import load_bert_data
model = load_bert_data(model,'/home/xiaoguzai/数据集/Nezha-Base/pytorch_model.bin')
#from loader_pretrain_weights import load_bert_data
#model = load_bert_data(model,'/home/xiaoguzai/数据集/剧本角色识别/begin.pth') # 仅加载参数
optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5)
#145个文件之前学习率为2e-5，从145个文件之后学习率变成4e-5，学完150个文件之后学习train.txt和test.txt的对应文件内容
#此时学习率为6e-5
loss_func = torch.nn.CrossEntropyLoss()


---__init__ Nezha
!!!load Pytorch model!!!
Done loading 202 NEZHA weights from: /home/xiaoguzai/数据集/Nezha-Base/pytorch_model.bin. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	cls.seq_relationship.weight
	cls.seq_relationship.bias
	cls.predictions.decoder.weight
	bert.pooler.dense.bias
	bert.pooler.dense.weight


In [5]:
import csv
train = pd.read_csv('/home/xiaoguzai/代码/剧本角色情感识别/数据集/train_dataset_v2.tsv',sep='\t')
content_dict = {}

for index in range(len(train['content'])):
    currentindex = train['id'][index]
    currentindex = currentindex.split('_')
    str1 = currentindex[0]
    while len(str1) < 5:
        str1 = '0'+str1
    str4 = currentindex[3]
    while len(str4) < 4:
        str4 = '0'+str4
    resultindex = str1+'_'+currentindex[1]+'_'+currentindex[2]+'_'+str4
    content_dict[resultindex] = train['content'][index]

def sortedDictValues(adict):
    keys = adict.keys()
    keys = sorted(keys)
    new_content_dict = {}
    for data in keys:
        new_content_dict[data] = content_dict[data]
    return new_content_dict
new_content_dict = sortedDictValues(content_dict)

inv_new_content_dict = {}
for data in new_content_dict:
    inv_new_content_dict[new_content_dict[data]] = data

keys = new_content_dict.keys()
keys = list(keys)
dict_keys_index = {}
for index in range(len(keys)):
    dict_keys_index[keys[index]] = index
    
content = []
for index in range(len(train['content'])):
    current_text = train['content'][index]
    
    current_str_index = inv_new_content_dict[current_text]
    current_key_index = dict_keys_index[current_str_index]
    while current_key_index != -1 and new_content_dict[keys[current_key_index]] == current_text:
        current_key_index = current_key_index-1
    if current_key_index == -1:
        pre_content = ''
    else:
        pre_content = new_content_dict[keys[current_key_index]]
    new_str_index = keys[current_key_index]
    current_str_index = current_str_index.split('_')
    new_str_index = new_str_index.split('_')
    if current_str_index[0] != new_str_index[0]:
        pre_content = ''
    
    current_str_index = inv_new_content_dict[current_text]
    current_key_index = dict_keys_index[current_str_index]
    total_key_index = len(dict_keys_index)-1
    while current_key_index!= total_key_index and new_content_dict[keys[current_key_index]] == current_text:
        current_key_index = current_key_index+1
    if current_key_index == total_key_index:
        after_content = ''
    else:
        after_content = new_content_dict[keys[current_key_index]]
    new_str_index= keys[current_key_index]
    current_str_index = current_str_index.split('_')
    new_str_index = new_str_index.split('_')
    if current_str_index[0] != new_str_index[0]:
        after_content = ''
    
    content.append(pre_content+train['content'][index]+after_content)

In [6]:
len(content)

42790

In [7]:
test = pd.read_csv('/home/xiaoguzai/代码/剧本角色情感识别/数据集/test_dataset.tsv',sep='\t')
content_dict = {}

for index in range(len(test['content'])):
    currentindex = test['id'][index]
    currentindex = currentindex.split('_')
    str1 = currentindex[0]
    while len(str1) < 5:
        str1 = '0'+str1
    str4 = currentindex[3]
    while len(str4) < 4:
        str4 = '0'+str4
    resultindex = str1+'_'+currentindex[1]+'_'+currentindex[2]+'_'+str4
    content_dict[resultindex] = test['content'][index]

def sortedDictValues(adict):
    keys = adict.keys()
    keys = sorted(keys)
    new_content_dict = {}
    for data in keys:
        new_content_dict[data] = content_dict[data]
    return new_content_dict
new_content_dict = sortedDictValues(content_dict)

inv_new_content_dict = {}
for data in new_content_dict:
    inv_new_content_dict[new_content_dict[data]] = data

keys = new_content_dict.keys()
keys = list(keys)
dict_keys_index = {}
for index in range(len(keys)):
    dict_keys_index[keys[index]] = index
    
for index in range(len(test['content'])):
    current_text = test['content'][index]
    
    current_str_index = inv_new_content_dict[current_text]
    current_key_index = dict_keys_index[current_str_index]
    while current_key_index != -1 and new_content_dict[keys[current_key_index]] == current_text:
        current_key_index = current_key_index-1
    if current_key_index == -1:
        pre_content = ''
    else:
        pre_content = new_content_dict[keys[current_key_index]]
    new_str_index = keys[current_key_index]
    current_str_index = current_str_index.split('_')
    new_str_index = new_str_index.split('_')
    if current_str_index[0] != new_str_index[0]:
        pre_content = ''
    
    current_str_index = inv_new_content_dict[current_text]
    current_key_index = dict_keys_index[current_str_index]
    total_key_index = len(dict_keys_index)-1
    while current_key_index!= total_key_index and new_content_dict[keys[current_key_index]] == current_text:
        current_key_index = current_key_index+1
    if current_key_index == total_key_index:
        after_content = ''
    else:
        after_content = new_content_dict[keys[current_key_index]]
    new_str_index= keys[current_key_index]
    current_str_index = current_str_index.split('_')
    new_str_index = new_str_index.split('_')
    if current_str_index[0] != new_str_index[0]:
        after_content = ''
    
    content.append(pre_content+test['content'][index]+after_content)

In [8]:
len(content)

64166

In [9]:
from nezha_pretraining import random_mask
class ClassificationDataset(Dataset):
    def __init__(self,text,maxlen):
        self.text = text
        self.maxlen = maxlen
        token_data,token_id,segment_id,mask_id = [],[],[],[]
        #sequence填充可以最后统一实现
        for index in tqdm(range(len(self.text))):
        #!!!这里构成ClassificationDataset之中的tqdm还可以优化并行操作
            current_text = text[index]
            current_token = tokenizer.tokenize(current_text)
            current_token = ["[CLS]"]+current_token
            current_id = tokenizer.convert_tokens_to_ids(current_token)
            if len(current_token) > maxlen-1:
                current_token = current_token[:maxlen-1]
                current_id = current_id[:maxlen-1]
            current_token = current_token+["[SEP]"]
            current_id = current_id+tokenizer.convert_tokens_to_ids(["[SEP]"])
            token_data.append(current_token)
            token_id.append(current_id)
        begin = 5
        #从第5个数值开始标记
        self.input_id,self.label = nezha_pretraining_store_id_data(token_id,token_data,vocab_file,begin,vocab_size)
        self.input_id = sequence_padding(self.input_id,maxlen)
        self.label = sequence_padding(self.label,maxlen,padding=-100)
        self.tensors = [torch.tensor(self.input_id,dtype=torch.long).to(device),
                       torch.tensor(self.label,dtype=torch.long).to(device)]
        #tensor之后，清除原先的内存会导致无法训练，注意这里暂时不能够放入device之中
        
    def __len__(self):
        return len(self.tensors[0])
    
    def __getitem__(self,index):
        return tuple(tensor[index] for tensor in self.tensors)
    
    def sequence_padding(self,inputs,padding = 0):
        length = self.maxlen
        if len(inputs) > length:
            inputs = inputs[:length]
        outputs = []
        pad_width = (0,length-len(inputs))
        x = np.pad(inputs,pad_width,'constant',constant_values=padding)
        return x

In [10]:
content_maxlen = 200
content_dataset = ClassificationDataset(content,maxlen=content_maxlen)
content_data_iter = DataLoader(content_dataset,batch_size=32,shuffle=True)

100%|██████████| 64166/64166 [00:13<00:00, 4715.82it/s]


  0%|          | 0/64166 [00:00<?, ?it/s]

  0%|          | 0/64166 [00:00<?, ?it/s]

  0%|          | 0/64166 [00:00<?, ?it/s]

In [11]:
loss_func = torch.nn.CrossEntropyLoss()
#torch.cuda.set_device(0)
model = model.to(device)
model = nn.DataParallel(model)
model.train()
scaler = torch.cuda.amp.GradScaler()
best_loss = 999999999999
best_accuracy = 0.0
for epoch in range(100):
    print('epoch {}'.format(epoch+1))
    train_loss = 0
    train_acc = 0
    content_dataset_len = len(content_data_iter)
    for batch_token_ids,batch_labels in tqdm(content_data_iter,colour='blue'):
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()
        output = model(batch_token_ids)
        output = output.view(-1,vocab_size)
        batch_labels = batch_labels.view(-1)
        loss = loss_func(output,batch_labels)
        train_loss += loss
        pred = torch.max(output, 1)[1]
        train_correct = (pred == batch_labels).sum()
        train_acc += train_correct
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss/content_dataset_len, train_acc/content_dataset_len))
    current_loss = train_loss/content_dataset_len
    current_accuracy = train_acc/content_dataset_len
    if current_loss < best_loss and current_accuracy > best_accuracy:
        torch.save(model.state_dict(),'/home/xiaoguzai/数据集/剧本角色识别/'+'labeled_data+best_data.pth')
    if epoch%10 == 0:
        torch.save(model.state_dict(),'/home/xiaoguzai/数据集/剧本角色识别/'+'labeled_data+model_epoch='+str(epoch)+'.pth')


epoch 1


100%|██████████| 2006/2006 [08:40<00:00,  3.86it/s]


Train Loss: 2.266596, Acc: 191.444672
epoch 2


100%|██████████| 2006/2006 [08:38<00:00,  3.87it/s]


Train Loss: 1.697758, Acc: 221.997009
epoch 3


...

Train Loss: 0.009750, Acc: 350.336487
epoch 99


100%|██████████| 2006/2006 [08:46<00:00,  3.81it/s]


Train Loss: 0.009621, Acc: 350.363922
epoch 100


100%|██████████| 2006/2006 [09:02<00:00,  3.70it/s]


Train Loss: 0.009180, Acc: 350.431702


In [12]:
for epoch in range(100,200):
    print('epoch {}'.format(epoch+1))
    train_loss = 0
    train_acc = 0
    content_dataset_len = len(content_data_iter)
    for batch_token_ids,batch_labels in tqdm(content_data_iter,colour='blue'):
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()
        output = model(batch_token_ids)
        output = output.view(-1,vocab_size)
        batch_labels = batch_labels.view(-1)
        loss = loss_func(output,batch_labels)
        train_loss += loss
        pred = torch.max(output, 1)[1]
        train_correct = (pred == batch_labels).sum()
        train_acc += train_correct
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss/content_dataset_len, train_acc/content_dataset_len))
    current_loss = train_loss/content_dataset_len
    current_ac
    curacy = train_acc/content_dataset_len
    if current_loss < best_loss and current_accuracy > best_accuracy:
        torch.save(model.state_dict(),'/home/xiaoguzai/数据集/剧本角色识别/'+'labeled_data+best_data.pth')
    if epoch%10 == 0:
        torch.save(model.state_dict(),'/home/xiaoguzai/数据集/剧本角色识别/'+'labeled_data+model_epoch='+str(epoch)+'.pth')


epoch 101


100%|██████████| 2006/2006 [08:14<00:00,  4.06it/s]


Train Loss: 0.009621, Acc: 350.381348
epoch 102


100%|██████████| 2006/2006 [08:10<00:00,  4.09it/s]


Train Loss: 0.009145, Acc: 350.419739
epoch 103


...

Train Loss: 0.005557, Acc: 350.789642
epoch 199


100%|██████████| 2006/2006 [09:03<00:00,  3.69it/s]


Train Loss: 0.005803, Acc: 350.751740
epoch 200


100%|██████████| 2006/2006 [09:03<00:00,  3.69it/s]


Train Loss: 0.005262, Acc: 350.817566


In [ ]:
for epoch in range(200,301):
    print('epoch {}'.format(epoch+1))
    train_loss = 0
    train_acc = 0
    content_dataset_len = len(content_data_iter)
    for batch_token_ids,batch_labels in tqdm(content_data_iter,colour='blue'):
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = batch_labels.to(device)
        optimizer.zero_grad()
        output = model(batch_token_ids)
        output = output.view(-1,vocab_size)
        batch_labels = batch_labels.view(-1)
        loss = loss_func(output,batch_labels)
        train_loss += loss
        pred = torch.max(output, 1)[1]
        train_correct = (pred == batch_labels).sum()
        train_acc += train_correct
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    print('Train Loss: {:.6f}, Acc: {:.6f}'.format(train_loss/content_dataset_len, train_acc/content_dataset_len))
    current_loss = train_loss/content_dataset_len
    current_accuracy = train_acc/content_dataset_len
    if current_loss < best_loss and current_accuracy > best_accuracy:
        torch.save(model.state_dict(),'/home/xiaoguzai/数据集/剧本角色识别/'+'labeled_data+best_data.pth')
    if epoch%10 == 0:
        torch.save(model.state_dict(),'/home/xiaoguzai/数据集/剧本角色识别/'+'labeled_data+model_epoch='+str(epoch)+'.pth')
